In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.simplefilter('ignore')

import gc

from os import path
import sys
sys.path.append(path.abspath('..'))

In [3]:
import os
import cv2
import torch
import pandas as pd
from PIL import Image
from pprint import pprint
import albumentations as albu
from albumentations.pytorch import ToTensorV2

from src.lightning_module import PosterModule

In [4]:
DEVICE = 'cuda:0'

In [5]:
# Загружаем модель

checkpoint_name = '../model/model.ckpt'

module = PosterModule.load_from_checkpoint(checkpoint_name)

_ = module.eval()
_ = module.to(DEVICE)

In [6]:
# Загружаем данные
df = pd.read_csv( '../data/df_test.csv')
names = list(df.columns[1:])
names
df.head()

['agriculture',
 'artisinal_mine',
 'bare_ground',
 'blooming',
 'blow_down',
 'clear',
 'cloudy',
 'conventional_mine',
 'cultivation',
 'habitation',
 'haze',
 'partly_cloudy',
 'primary',
 'road',
 'selective_logging',
 'slash_burn',
 'water']

,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,train_1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
1,train_3,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
2,train_5,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1
3,train_9,1,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0
4,train_12,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [7]:
# препроцессинг данных 
img_height = 224
img_width = 224
preprocess = albu.Compose(
        [
            albu.Resize(height=img_height, width=img_width),
            albu.Normalize(),
            ToTensorV2(),
        ]
)

In [9]:
idx = 50
image_name = os.path.join('../data', 'train-jpg', f'{df["image_name"][idx]}.jpg')
image = cv2.imread(image_name)[..., ::-1]
Image.fromarray(image)

In [10]:
procecces_image = preprocess(image=image)['image']
with torch.no_grad():
    scores = torch.sigmoid(module(procecces_image[None].to(DEVICE)))[0].cpu().numpy()
    
pprint({n:s for s, n in zip(scores, names)})

{'agriculture': 0.00792668,
 'artisinal_mine': 2.3634935e-07,
 'bare_ground': 0.00016628944,
 'blooming': 0.0054796925,
 'blow_down': 0.00018658942,
 'clear': 0.9998331,
 'cloudy': 2.3739989e-05,
 'conventional_mine': 7.775082e-08,
 'cultivation': 0.00023646498,
 'habitation': 0.00036585846,
 'haze': 7.554709e-05,
 'partly_cloudy': 0.00013050124,
 'primary': 0.9996427,
 'road': 0.0019264717,
 'selective_logging': 0.00019917286,
 'slash_burn': 7.918312e-06,
 'water': 0.00893271}
